In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /home/sandheepgopinath/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [7]:
## Tensor flow has only train and test datasets
#An arbitary percentage of train can be used to split into validation

mnist_train, mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [13]:
mnist_info.splits['train'].num_examples

60000

In [15]:
n_validation=0.1*mnist_info.splits['train'].num_examples

#Here there might be floating point numbers

n_validation=tf.cast(n_validation,tf.int64)

In [19]:
n_test=mnist_info.splits['test'].num_examples
n_test=tf.cast(n_test,tf.int64)

In [20]:
# Scaling an images to 0,1 range. Hence it has to be in Float 

def scale(image, label):
    image=tf.cast(image,tf.float32)
    image/=255. # The dot specifies that we need the output in Float
    return image, label
#Here the function only transforms as it takes and returns the same things


In [22]:
# THe map function in TF can be used to apply a custom transformation, but it needs an image and label


mnist_train_scaled=mnist_train.map(scale)
mnist_test_scaled=mnist_test.map(scale)

## Shuffling the data 


In [37]:
# Shuffling helps if the data is ordered.

def shuffleAndSplit(Train_data,BUFFER_SIZE,BATCH_SIZE,Split_percentage,Test_data,Batch=False,):
    shuffled_data=Train_data.shuffle(BUFFER_SIZE)
    validation_data=shuffled_data.take(Split_percentage)
    train_data=shuffled_data.skip(Split_percentage)
    if Batch==True:
        train_data=train_data.batch(BATCH_SIZE)
        validation_data=validation_data.batch(BATCH_SIZE)
        test_data=Test_data.batch(BATCH_SIZE)
    return train_data,validation_data,test_data

In [39]:
train,test,validation=shuffleAndSplit(mnist_train_scaled,10000,100,n_validation,mnist_test_scaled,Batch=True)

## Splitting the input and target


In [45]:
validation_input,validation_targets=next(iter(validation))

# Creating a Model


In [49]:
model=tf.keras.Sequential()

#Adding layers to the model

model.add(tf.keras.layers.Flatten(input_shape=(28,28,1)))
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])



In [ ]:
model.fit(train,epochs=10,validation_data=(validation_input,validation_targets),validation_steps=1,verbose=1)

Epoch 1/10
540/540 [==============================] - 5s 9ms/step - loss: 0.2694 - accuracy: 0.9181 - val_loss: 0.1024 - val_accuracy: 0.9600
Epoch 2/10
540/540 [==============================] - 3s 5ms/step - loss: 0.0997 - accuracy: 0.9698 - val_loss: 0.1209 - val_accuracy: 0.9700
Epoch 3/10
489/540 [==========================>...] - ETA: 0s - loss: 0.0658 - accuracy: 0.9797